In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [2]:
df_history = pd.read_csv("C:/Users/felix.oechslein/Desktop/Analysen Elphi, Mai/verarbeitete_daten/"
                                    "Elphi_history_ersteBereinigung.csv")

In [3]:
df_relevante_iids = pd.read_csv("C:/Users/felix.oechslein/Desktop/Analysen Elphi, Mai/verarbeitete_daten/" +
                                    "Elphi_transactions_zweiteBereinigung_iids.csv")

##### Relevante Kunden und Hist mergen

In [4]:
df_h = pd.merge(df_history, df_relevante_iids, on = ["iid"])
df_h.head()

,iid,attrtype,akey,date,text,month_count
0,163094976,HY,248941187,2022-04-12 22:31:07,|msg=2122 NL Konzerte 220412|subject=Musikfest...,0
1,163094976,HY,250740360,2022-04-21 13:17:09,|msg=2122 NL Konzerte 220421|subject=Sommer-Pr...,0
2,163094976,NO,250740360,2022-04-24 14:57:59,|msg=2122 NL Konzerte 220421|userAgent=Firefox...,0
3,163094976,HY,252063005,2022-04-26 17:12:51,|msg=2122 NL Konzerte 220426|subject=Open+Air+...,0
4,163094976,NO,252063005,2022-04-26 17:13:45,|msg=2122 NL Konzerte 220426|userAgent=Chrome ...,0


##### Erstellung des "df_neueSpalte" für die Betrachtung sämtlicher Infos zwischen den "|"

In [5]:
neueSpalten = [ line.strip().split('|') for line in df_h["text"] ]

##### Ein neues DataFrame df_user_agent und eine neue Spalte "User agent", die noch sämtliche Informationen danach enthält

In [19]:
df_user_agent = pd.DataFrame()
df_user_agent["User Agent"] = df_h["text"].str.extract(r'userAgent=(.*)', expand=False)
df_user_agent["User Agent"] = df_user_agent["User Agent"].apply(lambda zeile: zeile.split("|")[0] if isinstance(zeile, str) else zeile)

In [20]:
df_user_agent.head()

,User Agent
0,NaN
1,NaN
2,Firefox 99.0 (Windows 7 / Personal computer)
3,NaN
4,Chrome Mobile 100.0 (Android / Smartphone)


##### Ergänzung duch Infos für welche Datenpunkte Probleme bei der Genauigkeit der Öffnungsrate besteht

In [15]:
df_user_agent["User Agent"].fillna("unknown", inplace=True)
df_user_agent_aufgeteilt = df_user_agent

# Spalten mit Ja/Nein-Werten erstellen
df_user_agent_aufgeteilt["unknown"] = df_user_agent['User Agent'].str.contains('unknown', case=False)
#df_user_agent_aufgeteilt['iOS-Nutzer'] = df_user_agent['User Agent'].str.contains('iOS', case=False)
#df_user_agent_aufgeteilt['OS-X-Nutzer'] = df_user_agent['User Agent'].str.contains('OS X|Macintosh', case=False)
df_user_agent_aufgeteilt['Apple-Mail-Nutzer'] = df_user_agent['User Agent'].str.contains('Apple Mail', case=False)
df_user_agent_aufgeteilt['Gmail-Nutzer'] = df_user_agent['User Agent'].str.contains('Gmail', case=False)
df_user_agent_aufgeteilt['Proxy-Server'] = df_user_agent['User Agent'].str.contains('Proxy', case=False)
df_user_agent_aufgeteilt.head()

,User Agent,unknown,Apple-Mail-Nutzer,Gmail-Nutzer,Proxy-Server
0,unknown,True,False,False,False
1,unknown,True,False,False,False
2,Firefox 99.0 (Windows 7 / Personal computer),False,False,False,False
3,unknown,True,False,False,False
4,Chrome Mobile 100.0 (Android / Smartphone),False,False,False,False


In [18]:
for col in df_user_agent.columns[1:]:
    print(df_user_agent[col].value_counts())

True     167763
False     45171
Name: unknown, dtype: int64
False    208894
True       4040
Name: Apple-Mail-Nutzer, dtype: int64
False    209278
True       3656
Name: Gmail-Nutzer, dtype: int64
False    209278
True       3656
Name: Proxy-Server, dtype: int64
